In [1]:
import numpy as np
import pandas as pd
import impyute
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('Pollution_data.csv')
states = ['Delhi', 'West Bengal', 'Haryana']
df = df[df['state'].isin(states)]
# df = df[df['state'] == 'Delhi' or df['state'] == 'Uttarakhand' or df['state'] ==]
df.drop_duplicates(inplace=True)
drop_col = list(df.columns)
drop_col.remove('state')
drop_col.remove('rspm')
drop_col.remove('spm')
drop_col.remove('so2')
drop_col.remove('no2')

df.drop(columns=drop_col, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,state,so2,no2,rspm,spm
0,Delhi,8.6,16.2,NaN,323.0
1,Delhi,6.5,9.3,NaN,488.0
2,Delhi,25.8,10.4,NaN,487.0
3,Delhi,0.5,2.8,NaN,528.0
4,Delhi,2.5,12.8,NaN,506.0


In [3]:
df.to_csv('original.csv')

In [4]:
df.shape

(34417, 5)

In [5]:
test = df.dropna(axis=0, how='any')
test.reset_index(drop=True, inplace=True)
test.shape

(15328, 5)

In [6]:
def AQI_cal(test):  
    def cal_SOi(so2):
        si=0
        if (so2<=40):
         si= so2*(50/40)
        elif (so2>40 and so2<=80):
         si= 50+(so2-40)*(50/40)
        elif (so2>80 and so2<=380):
         si= 100+(so2-80)*(100/300)
        elif (so2>380 and so2<=800):
         si= 200+(so2-380)*(100/420)
        elif (so2>800 and so2<=1600):
         si= 300+(so2-800)*(100/800)
        elif (so2>1600):
         si= 400+(so2-1600)*(100/800)
        return si
    test['SOi']=test['so2'].apply(cal_SOi)

    def cal_Noi(no2):
        ni=0
        if(no2<=40):
         ni= no2*50/40
        elif(no2>40 and no2<=80):
         ni= 50+(no2-40)*(50/40)
        elif(no2>80 and no2<=180):
         ni= 100+(no2-80)*(100/100)
        elif(no2>180 and no2<=280):
         ni= 200+(no2-180)*(100/100)
        elif(no2>280 and no2<=400):
         ni= 300+(no2-280)*(100/120)
        else:
         ni= 400+(no2-400)*(100/120)
        return ni
    test['NOi']=test['no2'].apply(cal_Noi)

    def cal_RSPMi(rspm):
        rpi=0
        if(rspm<=100):
         rpi = rspm
        elif(rspm>=101 and rspm<=150):
         rpi= 101+(rspm-101)*((200-101)/(150-101))
        elif(rspm>=151 and rspm<=350):
         rpi= 201+(rspm-151)*((300-201)/(350-151))
        elif(rspm>=351 and rspm<=420):
         rpi= 301+(rspm-351)*((400-301)/(420-351))
        elif(rspm>420):
         rpi= 401+(rspm-420)*((500-401)/(420-351))
        return rpi
    test['RSPMi']=test['rspm'].apply(cal_RSPMi)

    def cal_SPMi(spm):
        spi=0
        if(spm<=50):
         spi=spm*50/50
        elif(spm>50 and spm<=100):
         spi=50+(spm-50)*(50/50)
        elif(spm>100 and spm<=250):
         spi= 100+(spm-100)*(100/150)
        elif(spm>250 and spm<=350):
         spi=200+(spm-250)*(100/100)
        elif(spm>350 and spm<=430):
         spi=300+(spm-350)*(100/80)
        else:
         spi=400+(spm-430)*(100/430)
        return spi
    test['SPMi']=test['spm'].apply(cal_SPMi)

    def cal_aqi(si,ni,rspmi,spmi):
        aqi=0
        if(si>ni and si>rspmi and si>spmi):
         aqi=si
        if(ni>si and ni>rspmi and ni>spmi ):
         aqi=ni
        if(rspmi>si and rspmi>ni and rspmi>spmi ):
         aqi=rspmi
        if(spmi>si and spmi>ni and spmi>rspmi):
         aqi=spmi
        return aqi
    test['AQI']=test.apply(lambda x:cal_aqi(x['SOi'],x['NOi'],x['RSPMi'],x['SPMi']),axis=1)
    def AQI_Range(x):
        if x<=50:
            return "Good"
        elif x>50 and x<=100:
            return "Moderate"
        elif x>100 and x<=200:
            return "Poor"
        elif x>200 and x<=300:
            return "Unhealthy"
        elif x>300 and x<=400:
            return "Very unhealthy"
        elif x>400:
            return "Hazardous"

    test['AQI_Range'] = test['AQI'] .apply(AQI_Range)
    test.head()
AQI_cal(test)

In [7]:
test.to_csv('withoutNull.csv')

In [8]:
#fitting
Withoutnull = pd.read_csv("withoutNull.csv")
AQI_cal(Withoutnull)
from sklearn.neighbors import KNeighborsClassifier as knn
Delhi_cl=knn(n_neighbors=5)
WB_cl=knn(n_neighbors=5)
Haryana_cl=knn(n_neighbors=5)

Delhi_wn = Withoutnull[Withoutnull.state=="Delhi"]
Haryana_wn = Withoutnull[Withoutnull.state=="Haryana"]
WB_wn = Withoutnull[Withoutnull.state=="West Bengal"]

In [9]:
Delhi_cl.fit(Delhi_wn[['so2','no2','rspm','spm']],Delhi_wn["AQI_Range"])
Haryana_cl.fit(Haryana_wn[['so2','no2','rspm','spm']],Haryana_wn["AQI_Range"])
WB_cl.fit(WB_wn[['so2','no2','rspm','spm']],WB_wn["AQI_Range"])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [20]:
#KNN Imputation
from fancyimpute import KNN
ds = df[df['state']==states[2]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()
knn_imputer = KNN()
knn_res = knn_imputer.fit_transform(ds)
knn_res = pd.DataFrame(knn_res)
knn_res.columns = ds.columns
knn_res.head()

Imputing row 1/3415 with 2 missing, elapsed time: 2.203
Imputing row 101/3415 with 2 missing, elapsed time: 2.211
Imputing row 201/3415 with 1 missing, elapsed time: 2.219
Imputing row 301/3415 with 1 missing, elapsed time: 2.227
Imputing row 401/3415 with 0 missing, elapsed time: 2.231
Imputing row 501/3415 with 0 missing, elapsed time: 2.231
Imputing row 601/3415 with 0 missing, elapsed time: 2.231
Imputing row 701/3415 with 0 missing, elapsed time: 2.235
Imputing row 801/3415 with 0 missing, elapsed time: 2.235
Imputing row 901/3415 with 2 missing, elapsed time: 2.239
Imputing row 1001/3415 with 2 missing, elapsed time: 2.247
Imputing row 1101/3415 with 0 missing, elapsed time: 2.251
Imputing row 1201/3415 with 0 missing, elapsed time: 2.255
Imputing row 1301/3415 with 0 missing, elapsed time: 2.255
Imputing row 1401/3415 with 0 missing, elapsed time: 2.259
Imputing row 1501/3415 with 0 missing, elapsed time: 2.263
Imputing row 1601/3415 with 4 missing, elapsed time: 2.263
Imputing 

,so2,no2,rspm,spm
0,43.2,24.100000,73.356759,330.966334
1,41.8,24.000000,78.357400,362.720456
2,45.7,24.400000,84.000045,228.000000
3,42.8,22.000000,78.648075,364.839576
4,46.8,32.535293,262.309674,279.876996


In [21]:
AQI_cal(knn_res)

In [22]:
knn_res.to_csv(states[2] + '_knn.csv')

In [29]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

ind = 2

ds = df[df['state']==states[ind]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()

MICE_imputer = IterativeImputer()
MICE_res = MICE_imputer.fit_transform(ds)
MICE_res = pd.DataFrame(MICE_res)
MICE_res.columns=ds.columns
MICE_res.head()

,so2,no2,rspm,spm
0,43.2,24.100000,253.928668,349.712243
1,41.8,24.000000,249.362485,346.631985
2,45.7,24.400000,175.828143,228.000000
3,42.8,22.000000,251.540430,338.616946
4,46.8,39.717027,273.115362,432.709708


In [30]:
MICE_res.isnull().sum()
AQI_cal(MICE_res)

In [31]:
MICE_res.to_csv(states[ind] + '_MICE.csv')

In [48]:
#KNN Imputation
from fancyimpute import SimpleFill

ind = 0
ds = df[df['state']==states[ind]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()

mean_res = SimpleFill('mean')
mean_res = mean_res.fit_transform(ds)
mean_res = pd.DataFrame(mean_res)
mean_res.columns=ds.columns
mean_res.head()

,so2,no2,rspm,spm
0,8.6,16.2,196.639771,323.0
1,6.5,9.3,196.639771,488.0
2,25.8,10.4,196.639771,487.0
3,0.5,2.8,196.639771,528.0
4,2.5,12.8,196.639771,506.0


In [49]:
AQI_cal(mean_res)

In [50]:
mean_res.to_csv(states[ind] + '_mean.csv')

In [58]:
#Random Forest
from missingpy import MissForest
ind = 0
ds = df[df['state']==states[ind]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()

imputer = MissForest()
Random_forest = imputer.fit_transform(ds)
Random_forest = pd.DataFrame(Random_forest)
Random_forest.columns=ds.columns
Random_forest.head()

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


,so2,no2,rspm,spm
0,8.6,16.2,116.32,323.0
1,6.5,9.3,220.54,488.0
2,25.8,10.4,248.19,487.0
3,0.5,2.8,189.01,528.0
4,2.5,12.8,177.11,506.0


In [59]:
AQI_cal(Random_forest)

In [60]:
Random_forest.to_csv(states[ind] + '_RF.csv')

In [64]:
# Linear Interpolation
ind = 0
ds = df[df['state']==states[ind]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()

LI_res=ds.interpolate(method ='linear', limit_direction ='both') 
LI_res = pd.DataFrame(LI_res)
LI_res.columns=ds.columns
AQI_cal(LI_res)
LI_res.to_csv(states[ind]+"LI.csv")

In [67]:
#EM
import impyute as impy
ind = 0
ds = df[df['state']==states[ind]]
new_ds = ds[["so2", "no2","rspm","spm"]].values
# ds.head()
ds.drop(columns=['state'], inplace=True)
X = ds.iloc[:,:]
# ds.isnull().sum()

EM = impy.imputation.cs.em(new_ds, loops=50)
EM= pd.DataFrame(EM)
EM.columns=ds.columns
AQI_cal(EM)
EM.to_csv(states[ind]+"EM.csv")

In [144]:
EM.isna().sum()

so2     0
no2     0
rspm    0
spm     0
dtype: int64

In [145]:
EM.shape

(3415, 4)

In [146]:
ds.shape

(3415, 4)

In [163]:
#fitting
from sklearn.neighbors import KNeighborsClassifier as knn
classifier=knn(n_neighbors=5)


In [164]:
Haryana_knn = pd.read_csv("Haryana_knn.csv")
Haryana_MICE = pd.read_csv("Haryana_MICE.csv")
Haryana_median = pd.read_csv("Haryana_median.csv")

In [165]:
withoutNull= pd.read_csv("withoutNull.csv")

In [166]:
withoutNull.head()

,Unnamed: 0,state,so2,no2,rspm,spm,SOi,NOi,RSPMi,SPMi,AQI,AQI_Range
0,0,Delhi,11.0,39.0,184.0,405.0,13.75,48.75,217.417085,368.75,368.75,Very unhealthy
1,1,Delhi,12.0,42.0,190.0,429.0,15.00,52.50,220.402010,398.75,398.75,Very unhealthy
2,2,Delhi,14.0,41.0,187.0,304.0,17.50,51.25,218.909548,254.00,254.00,Unhealthy
3,3,Delhi,13.0,39.0,192.0,323.0,16.25,48.75,221.396985,273.00,273.00,Unhealthy
4,4,Delhi,12.0,36.0,170.0,325.0,15.00,45.00,210.452261,275.00,275.00,Unhealthy


In [167]:
X = withoutNull[['so2','no2','rspm','spm']]
y = withoutNull['AQI_Range']

In [168]:
classifier.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [169]:
Haryana_knn.head()

,Unnamed: 0,so2,no2,rspm,spm
0,0,43.2,24.100000,73.356759,330.966334
1,1,41.8,24.000000,78.357400,362.720456
2,2,45.7,24.400000,84.000045,228.000000
3,3,42.8,22.000000,78.648075,364.839576
4,4,46.8,32.535293,262.309674,279.876996


In [170]:
AQI_cal(Haryana_knn)
AQI_cal(Haryana_MICE)
AQI_cal(Haryana_median)
Haryana_median.head()

,Unnamed: 0,so2,no2,rspm,spm,SOi,NOi,RSPMi,SPMi,AQI,AQI_Range
0,0,43.2,24.10,149.165,261.0,54.000,30.1250,198.312959,211.000000,211.000000,Unhealthy
1,1,41.8,24.00,149.165,261.0,52.250,30.0000,198.312959,211.000000,211.000000,Unhealthy
2,2,45.7,24.40,149.165,228.0,57.125,30.5000,198.312959,185.333333,198.312959,Poor
3,3,42.8,22.00,149.165,261.0,53.500,27.5000,198.312959,211.000000,211.000000,Unhealthy
4,4,46.8,21.15,149.165,261.0,58.500,26.4375,198.312959,211.000000,211.000000,Unhealthy


In [171]:
X1=Haryana_knn.iloc[:,1:5]
X2=Haryana_MICE.iloc[:,1:5]
X3=Haryana_median.iloc[:,1:5]
y1=Haryana_knn.iloc[:,10]
y2=Haryana_MICE.iloc[:,10]
y3=Haryana_median.iloc[:,10]

In [172]:
p1=classifier.predict(X1)
p2=classifier.predict(X2)
p3=classifier.predict(X3)

In [173]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [174]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y1,p1)

In [175]:
actual = y1
pred = p1
accuracy = accuracy_score(actual, pred) * 100
precision = precision_score(actual, pred,average='weighted') * 100
recall = recall_score(actual, pred,average='weighted') * 100
f1 = f1_score(actual, pred,average='weighted')
print ('MICE\nAccuracy is {:.4f}%\n\Precision is {:.4f}%\n\Recall is {:.4f}%\n\F1 Score is {:.4f}'.format(accuracy, precision, recall, f1))

MICE
Accuracy is 97.1303%
\Precision is 97.1565%
\Recall is 97.1303%
\F1 Score is 0.9712


In [176]:
actual = y3
pred = p3
accuracy = accuracy_score(actual, pred) * 100
precision = precision_score(actual, pred,average='weighted') * 100
recall = recall_score(actual, pred,average='weighted') * 100
f1 = f1_score(actual, pred,average='weighted')
print ('Median\nAccuracy is {:.4f}%\n\Precision is {:.4f}%\n\Recall is {:.4f}%\n\F1 Score is {:.4f}'.format(accuracy, precision, recall, f1))

Median
Accuracy is 98.1552%
\Precision is 98.1574%
\Recall is 98.1552%
\F1 Score is 0.9815
